PART 1 
Dataset1
Step 1:
对每个病人，读取 CT 图像和 segmentation mask，
自动对齐 Z 坐标并裁剪，
得到 image_3d 和 mask_3d，用于提 ROI 特征。

In [6]:
import os, pydicom, numpy as np, SimpleITK as sitk

# 从一个 DICOM 文件夹读取为 3D 图像（SimpleITK 对象）
def read_dicom_series(folder_path):
    reader = sitk.ImageSeriesReader()
    dicom_names = reader.GetGDCMSeriesFileNames(folder_path)
    reader.SetFileNames(dicom_names)
    return reader.Execute()

# 从 segmentation 文件中提取它引用的 CT 图像的 Series UID
def get_series_uid_from_seg(seg_path):
    return pydicom.dcmread(seg_path).ReferencedSeriesSequence[0].SeriesInstanceUID

# 递归查找给定目录下，是否有某个 .dcm 文件的 SeriesInstanceUID 与目标 UID 匹配
def find_ct_folder_by_uid(folder, uid):
    for root, _, files in os.walk(folder):
        for f in files:
            if f.lower().endswith(".dcm"):
                try:
                    if pydicom.dcmread(os.path.join(root, f), stop_before_pixels=True).SeriesInstanceUID == uid:
                        return root
                except: continue
    return None

# 计算 3D 图像中每一层的真实空间 Z 坐标，考虑方向矩阵、origin 和 spacing
def get_z_coords(sitk_img):
    sz, sp, org, dir3 = sitk_img.GetSize(), sitk_img.GetSpacing(), sitk_img.GetOrigin(), sitk_img.GetDirection()
    return [org[2] + i * sp[2] * dir3[8] for i in range(sz[2])]

# 将 segmentation 图像中 Z 坐标不在 CT 图中的那部分切掉，确保两者的 Z 层匹配
def align_segmentation_by_z(seg_img, seg_z, ct_z):
    zset = set(round(z, 1) for z in ct_z)
    return sitk.GetArrayFromImage(seg_img)[[i for i, z in enumerate(seg_z) if round(z, 1) in zset]]

# 将 CT 或 segmentation 图像重采样（resample）为统一 spacing（默认 1mm³），保证空间尺度一致。
def resample_to_spacing(image_sitk, new_spacing=(1.0, 1.0, 1.0), interpolator=sitk.sitkLinear):
    original_spacing = image_sitk.GetSpacing()
    original_size = image_sitk.GetSize()
    new_size = [
        int(round(original_size[i] * (original_spacing[i] / new_spacing[i])))
        for i in range(3)
    ]
    resampler = sitk.ResampleImageFilter()
    resampler.SetOutputSpacing(new_spacing)
    resampler.SetSize(new_size)
    resampler.SetOutputDirection(image_sitk.GetDirection())
    resampler.SetOutputOrigin(image_sitk.GetOrigin())
    resampler.SetInterpolator(interpolator)
    return resampler.Execute(image_sitk)

# 裁剪HU值范围并归一化到[0,1] (rescale)
def normalize_gray_roi_adaptive(image_np, mask_np, min_percentile=1, max_percentile=99):
    roi = image_np[mask_np > 0]
    if roi.size == 0:
        return np.zeros_like(image_np)  # 全部变0避免炸掉
    vmin, vmax = np.percentile(roi, [min_percentile, max_percentile])
    image_clipped = np.clip(image_np, vmin, vmax)
    return (image_clipped - vmin) / (vmax - vmin + 1e-6)

# 主函数：找到并读取CT和segmentation，resample,对齐z坐标，归一化灰度
def load_aligned_ct_and_mask(patient_folder, target_spacing=(1.5, 1.5, 1.5)):
    seg_path = next((os.path.join(root, f) for root, _, files in os.walk(patient_folder)
                     for f in files if "segmentation" in root.lower() and f.endswith(".dcm")), None)
    if not seg_path: return None, None, None

    uid = get_series_uid_from_seg(seg_path)
    ct_folder = find_ct_folder_by_uid(patient_folder, uid)
    if not ct_folder: return None, None, None

    # CT 和 segmentation 原始图像
    ct_img = read_dicom_series(ct_folder)
    seg_img = sitk.ReadImage(seg_path)

    # Resample to same spacing (1mm)
    ct_img = resample_to_spacing(ct_img, new_spacing=target_spacing)
    
    # 用 CT 图作为参考，resample segmentation
    resampler = sitk.ResampleImageFilter()
    resampler.SetReferenceImage(ct_img)
    resampler.SetInterpolator(sitk.sitkNearestNeighbor)
    seg_img = resampler.Execute(seg_img)

    # Z 坐标对齐
    ct_z = get_z_coords(ct_img)
    seg_z = get_z_coords(seg_img)
    mask = align_segmentation_by_z(seg_img, seg_z, ct_z)

    image = sitk.GetArrayFromImage(ct_img)
    image = normalize_gray_roi_adaptive(image, mask)

    # 最终输出
    return image, mask, target_spacing

In [7]:
base_dir = "../testdata/dataset1"

for patient_id in sorted(os.listdir(base_dir)):
    patient_path = os.path.join(base_dir, patient_id)
    if not os.path.isdir(patient_path):
        continue  # 跳过非文件夹

    image, mask, spacing = load_aligned_ct_and_mask(patient_path)

    if image is not None:
        roi = image * (mask > 0)
        print(f"{patient_id} ROI volume shape: {roi.shape}")
        # TODO: 可在这里添加特征提取、保存截图、收集 CSV 数据等
    else:
        print(f"{patient_id} 无法加载或对齐")

LUNG1-002 ROI volume shape: (222, 333, 333)
LUNG1-003 ROI volume shape: (214, 333, 333)
LUNG1-005 ROI volume shape: (182, 333, 333)


Step 2:手动提取特征

In [8]:
# 基于 marching cubes 算法提取肿瘤三维形状特征
import os
import numpy as np
import pandas as pd
import skimage as ski
from skimage import io, data, filters, measure, morphology
import matplotlib
import matplotlib.pyplot as plt
import nibabel as nib
import skimage.measure
from scipy.spatial.distance import  pdist

def extract_shape_features(mask, spacing=(1.0, 1.0, 1.0)):
    """
    输入：
        mask: 3D 二值数组，形如 (Z, H, W)，表示肿瘤区域
        spacing: 三元组 (sz, sy, sx)，表示每个 voxel 的空间大小（单位：mm）

    输出：
        一个字典，包含 volume, surface_area, max_diameter, compactness 等
    """

    # marching cubes: 生成肿瘤表面 mesh
    verts, faces, _, _ = measure.marching_cubes(mask, level=0.5, spacing=spacing)

    # 表面积（单位 mm²）
    surface_area = measure.mesh_surface_area(verts, faces)

    # 体积（单位 mm³）：mask 中为 1 的 voxel 总数 × voxel 体积
    voxel_volume = np.prod(spacing)
    volume = np.sum(mask > 0) * voxel_volume

    # 最大直径：点与点之间最大欧几里得距离（单位 mm）
    if len(verts) >= 2:
        max_diameter = pdist(verts).max()
    else:
        max_diameter = 0.0

    # 紧致度（表面积³ / 体积²）：越小越接近球体
    if volume > 0:
        compactness = (surface_area ** 3) / (volume ** 2)
    else:
        compactness = 0.0

    return {
        "volume_mm3": volume,
        "surface_mm2": surface_area,
        "max_diameter_mm": max_diameter,
        "compactness": compactness
    }


In [5]:
# 定义你要处理的病人 ID
patient_ids = ["LUNG1-005"]
base_dir = "../testdata/dataset1"  # 替换成你的真实路径

for patient_id in patient_ids:
    patient_path = os.path.join(base_dir, patient_id)

    image, mask, spacing = load_aligned_ct_and_mask(patient_path)
    if image is None or mask is None:
        print(f"{patient_id} 加载失败，跳过")
        continue

    features = extract_shape_features(mask, spacing)
    print(f"{patient_id} features:", features)


LUNG1-005 features: {'volume_mm3': 319146.75, 'surface_mm2': 65147.50703303793, 'max_diameter_mm': 243.24066726883004, 'compactness': 2714.6419241472163}


In [9]:
for patient_id in sorted(os.listdir(base_dir)):
    patient_path = os.path.join(base_dir, patient_id)
    if not os.path.isdir(patient_path):
        continue

    image, mask, spacing = load_aligned_ct_and_mask(patient_path)
    if image is None or mask is None:
        print(f"{patient_id} 加载失败，跳过")
        continue

    features = extract_shape_features(mask, spacing)
    print(f"{patient_id} features:", features)


LUNG1-002 features: {'volume_mm3': 515210.625, 'surface_mm2': 76237.54357674009, 'max_diameter_mm': 293.2212584678799, 'compactness': 1669.3102586993753}
LUNG1-003 features: {'volume_mm3': 369727.875, 'surface_mm2': 60603.535128362884, 'max_diameter_mm': 274.6759559269997, 'compactness': 1628.2814872188887}
LUNG1-005 features: {'volume_mm3': 319146.75, 'surface_mm2': 65147.50703303793, 'max_diameter_mm': 243.24066726883004, 'compactness': 2714.6419241472163}


In [11]:
def gray_level_cooccurrence_features(img, mask, levels=32):
    bin_img = (img * (levels - 1)).astype(np.uint8)
    glcm = _calculate_glcm2(bin_img, mask, levels)
    glcm = glcm / np.sum(glcm)

    ix = np.arange(1, levels+1)[:, None, None].astype(np.float64)
    iy = np.arange(1, levels+1)[None, :, None].astype(np.float64)

    ux = np.mean(glcm, axis=0, keepdims=True)
    uy = np.mean(glcm, axis=1, keepdims=True)
    sigma_x = np.std(glcm, axis=0, keepdims=True)
    sigma_y = np.std(glcm, axis=1, keepdims=True)

    # 下限保护
    sigma_x[sigma_x < 1e-3] = 1e-3
    sigma_y[sigma_y < 1e-3] = 1e-3

    features = {
        "contrast": np.mean(np.sum((ix - iy) ** 2 * glcm, axis=(0, 1))),
        "correlation": np.mean(np.sum((ix * iy * glcm - ux * uy) / (sigma_x * sigma_y + 1e-6), axis=(0, 1))),
        "dissimilarity": np.mean(np.sum(np.abs(ix - iy) * glcm, axis=(0, 1))),
        "homogeneity": np.mean(np.sum(glcm / (1 + np.abs(ix - iy)), axis=(0, 1))),
    }

    return features

In [12]:
def _calculate_glcm2(img, mask, nbins):
    out = np.zeros((nbins, nbins, 13))
    offsets = [
        (1, 0, 0), (0, 1, 0), (0, 0, 1),
        (1, 1, 0), (-1, 1, 0), (1, 0, 1),
        (-1, 0, 1), (0, 1, 1), (0, -1, 1),
        (1, 1, 1), (-1, 1, 1), (1, -1, 1), (1, 1, -1)
    ]
    matrix = np.array(img)
    matrix[mask <= 0] = nbins
    s = matrix.shape
    bins = np.arange(0, nbins + 1)

    for i, offset in enumerate(offsets):
        matrix1 = np.ravel(matrix[max(offset[0], 0):s[0]+min(offset[0], 0),
                                  max(offset[1], 0):s[1]+min(offset[1], 0),
                                  max(offset[2], 0):s[2]+min(offset[2], 0)])

        matrix2 = np.ravel(matrix[max(-offset[0], 0):s[0]+min(-offset[0], 0),
                                  max(-offset[1], 0):s[1]+min(-offset[1], 0),
                                  max(-offset[2], 0):s[2]+min(-offset[2], 0)])

        try:
            out[:, :, i] = np.histogram2d(matrix1, matrix2, bins=bins)[0]
        except Exception as e:
            print(f"GLCM histogram failed for offset {offset}: {e}")
            continue

    return out

In [70]:
import os
import pandas as pd

base_dir = "../testdata/dataset1"
all_features = []

for patient_id in sorted(os.listdir(base_dir)):
    # 跳过非“LUNG1-”命名的文件夹
    if not patient_id.startswith("LUNG1-"):
        continue

    patient_path = os.path.join(base_dir, patient_id)
    if not os.path.isdir(patient_path):
        continue

    image, mask, spacing = load_aligned_ct_and_mask(patient_path)

    if image is not None and mask is not None:
        image = normalize_gray_roi_adaptive(image, mask)
        glcm = gray_level_cooccurrence_features(image, mask)

        glcm["patient_id"] = patient_id
        all_features.append(glcm)
    else:
        print(f"{patient_id} 加载失败，跳过")

# 保存为 CSV 文件
df = pd.DataFrame(all_features)
df.to_csv("../result/glcm_features1.csv", index=False)
print("保存完成，共提取特征样本数:", len(df))

保存完成，共提取特征样本数: 5


In [13]:
import os
import pandas as pd
import numpy as np

base_dir = "../testdata/dataset1"
all_features = []

for patient_id in sorted(os.listdir(base_dir)):
    if not patient_id.startswith("LUNG1-"):
        continue

    patient_path = os.path.join(base_dir, patient_id)
    if not os.path.isdir(patient_path):
        continue

    image, mask, spacing = load_aligned_ct_and_mask(patient_path)
    if image is None or mask is None:
        print(f"{patient_id} 加载失败，跳过")
        continue

    image = normalize_gray_roi_adaptive(image, mask)
    shape_feats = extract_shape_features(mask, spacing)
    glcm_feats = gray_level_cooccurrence_features(image, mask)

    features = {"patient_id": patient_id}
    features.update(shape_feats)
    features.update(glcm_feats)
    all_features.append(features)

# 保存为 CSV
output_path = "../result/combined_features.csv"
os.makedirs(os.path.dirname(output_path), exist_ok=True)
df = pd.DataFrame(all_features)
df.to_csv("../result/features1.csv", index=False)
print(f"保存成功，共处理 {len(df)} 个病人")


保存成功，共处理 3 个病人


In [15]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import LeaveOneOut, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

# ==== Step 1: 读取数据 ====
# 替换为你本地的路径（如果不在当前目录）
features_df = pd.read_csv("../result/features1.csv")
clinical_df = pd.read_csv("../testdata/dataset1/clinical1.csv")

# 只保留必要的标签列
clinical_df = clinical_df[["PatientID", "deadstatus.event"]]

# ==== Step 2: 合并标签与影像特征 ====
df = features_df.merge(clinical_df, left_on="patient_id", right_on="PatientID")

# ==== Step 3: 准备模型输入 ====
X = df.drop(columns=["patient_id", "PatientID", "deadstatus.event"])
y = df["deadstatus.event"]

# ==== Step 4: 构建标准化 + 随机森林的模型流水线 ====
model = make_pipeline(StandardScaler(), RandomForestClassifier(n_estimators=100, random_state=42))

# ==== Step 5: Leave-One-Out 交叉验证（样本少时推荐） ====
loo = LeaveOneOut()
scores = cross_val_score(model, X, y, cv=loo, scoring="accuracy")

# ==== Step 6: 输出平均准确率 ====
print(f"平均准确率（Leave-One-Out）: {scores.mean():.3f}")


平均准确率（Leave-One-Out）: 1.000


In [ ]:
# RF建模，K-F交叉验证

import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler

# ==== 加载数据 ====
features_df = pd.read_csv("features1.csv")     # 替换为你 HPC 上路径
clinical_df = pd.read_csv("clinical1.csv")

# 只保留标签
labels_df = clinical_df[["PatientID", "deadstatus.event"]].dropna()

# ==== Stratified 5-Fold 交叉验证器 ====
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# ==== 模型：标准化 + 随机森林 ====
def evaluate_model(X, y, name="model"):
    model = make_pipeline(StandardScaler(), RandomForestClassifier(n_estimators=100, random_state=42))
    f1 = cross_val_score(model, X, y, cv=skf, scoring="f1")
    auc = cross_val_score(model, X, y, cv=skf, scoring="roc_auc")
    print(f"🧪 {name}：")
    print(f"  平均 F1: {f1.mean():.3f}  每折: {f1}")
    print(f"  平均 AUC: {auc.mean():.3f}  每折: {auc}")
    print()


In [ ]:
# 仅用影像
# 合并标签
df_a = features_df.merge(labels_df, left_on="patient_id", right_on="PatientID")
X_a = df_a.drop(columns=["patient_id", "PatientID", "deadstatus.event"])
y_a = df_a["deadstatus.event"]

evaluate_model(X_a, y_a, name="m_img")


In [ ]:
# 仅用临床
# 筛掉无标签或无关字段
clinical_features = clinical_df.drop(columns=["PatientID", "Survival.time", "deadstatus.event"])
df_b = labels_df.merge(clinical_features, left_on="PatientID", right_index=True)
X_b = df_b.drop(columns=["PatientID", "deadstatus.event"]).fillna(0)
y_b = df_b["deadstatus.event"]

evaluate_model(X_b, y_b, name="m_clinical")

In [ ]:
# 影像+临床
# 合并所有特征 + 标签
df_c = features_df.merge(clinical_df, left_on="patient_id", right_on="PatientID")
X_c = df_c.drop(columns=["patient_id", "PatientID", "Survival.time", "deadstatus.event"]).fillna(0)
y_c = df_c["deadstatus.event"]

evaluate_model(X_c, y_c, name="m_iac")